# 출처

* https://github.com/songys/Question_pair/blob/master/test.txt

In [1]:
import pandas as pd

In [2]:
from konlpy.tag import Mecab

In [1]:
import transformers
transformers.__version__
import gluonnlp

C:\Users\ysson\Anaconda3\lib\site-packages\mxnet\optimizer\optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


In [25]:
# 트레인 데이터 정제
kor_pair_train = pd.read_csv('kkangtongData/kor_pair_train.csv')
kor_pair_train = kor_pair_train[kor_pair_train['is_duplicate']==0]
kor_pair_train_final = kor_pair_train[['question1','question2']]
kor_pair_train_final.rename(columns={'question1':'Q','question2':'A'}, inplace = True)
kor_pair_train_final.reset_index(drop = True, inplace = True)
kor_pair_train_final.to_csv('kor_pair_train_final.csv')
kor = pd.read_csv('kor_pair_train_final.csv', index_col=0)

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [33]:
kor

,Q,A
0,2년동안 나 사랑해줘서 고맙다,2년여의 만남 사랑해 고마워
1,2년사귀다가 헤어젔는데,2년의 시간동안 사랑했는데
2,2년여의 만남,2년 동안 사궜어
3,3개월만에 연락이 왔어,3개월전에 헤어진 전남친이 문자했어
4,3년 사겼어,3년째 연애 중
...,...,...
4745,힘들고 지치네,힘들어.너무 힘들어
4746,힘들어 죽을거 같습니다,힘이 없어
4747,B형 남자랑 썸 탐.,B형 남자랑 썸을 타게 됨
4748,SD카드 안돼,SD카드 망가졌어


In [31]:
# 테스트 데이터 정제
kor_pair_test = pd.read_csv('kkangtongData/kor_Pair_test.csv')
kor_pair_test = kor_pair_test[kor_pair_test['is_duplicate']==0]
kor_pair_test_final = kor_pair_test[['question1','question2']]
kor_pair_test_final.rename(columns={'question1':'Q','question2':'A'}, inplace = True)
kor_pair_test_final.reset_index(drop = True, inplace = True)
kor_pair_test_final.to_csv('kor_pair_test_final.csv')


C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [32]:
kor_test = pd.read_csv('kor_pair_test_final.csv', index_col=0)
kor_test

,Q,A
0,2년 가량의 연애,2년 만났어
1,2년 만난 기념일이야,우리 벌써 2년 동안 사궜어
2,가까운 곳으로 나들이 갈까,바람 쐬러 어디 근교로 나갈까
3,간절히 원하면 이루어질까,꿈이 현실이었으면
4,강아지 귀여워 심쿵,강아지 밥 먹는거 귀여워
...,...,...
354,화해 어떻게 하지,싸웠는데 화해하고 싶어 어떻게 해?
355,황사땜에 눈아파,미세 먼지 때문에 눈이 아파
356,회사에 일이 너무 많아,일이 너무 많아서 해도해도 안 끝날 듯
357,회사에서 짤릴 필이 와,나 회사 짤릴 거 같은데 어떻게 만회하지


# AI hub V1

In [21]:
AI_hub_tweater_talk = pd.read_excel('kkangtongData/트위터_대화시나리오DB_2000Set.xlsx', engine='openpyxl', header=None, usecols='A:H')

In [24]:
AI_hub_tweater_talk.head()

,0,1,2,3,4,5,6,7
0,기계식 키보드 써보고 싶어요.,싼 것도 많던데 써보세요. 정말 좋아요.,싼 것도 있나요? 예전에 봤을 땐 다 10만 원이 넘던데요.,이거 쓰는데 예쁘고 딸깍거려요.,우와 진짜 예뻐요.,불 끄면 더 예뻐요.,저런 키보드 처음 봐요. 진짜 예쁘네요.,NaN
1,기계식 키보드 써보고 싶어요.,싼 것도 많던데 써보세요. 정말 좋아요.,싼 것도 있나요? 예전에 봤을 땐 다 10만 원이 넘던데요.,제 건 5만 원인가? 3만 원인가? 그래요. 근데 좋아요.,오 사볼 만한 가격이네요.,NaN,NaN,NaN
2,사실 여러 가지 글씨체가 있긴 한데 요즘 글씨를 안 쓰다 보니까 너무 맛 가버렸어요...,글씨 예쁘네요.,아니에요. 요즘 제 글씨 뭔지 너무 모르겠어요.,일정하지 않아도 저 정도면 너무 예쁜 거 아니에요? 난 악필이라서 노트 필기해도 볼...,나 노트필기 진짜 못해요. 맨날 하다가 포기해요.,저도 쓰다 보면 너무 구려서 포기하게 돼요. 그리고 공부도 포기하고요.,"맞아요. 아, 노트 필기해야지 하는데 글씨체 마음에 안 들면 바로 포기해요.",NaN
3,사실 여러 가지 글씨체가 있긴 한데 요즘 글씨를 안 쓰다 보니까 너무 맛 가버렸어요...,글씨 예뻐요.,우앵 아냐 시험 기간 끝나고 글씨 같은 거 안 쓰다 보니까 점점 망해가고 있어요.,나도요.,시험 끝나고 펜 잡는 거 너무 힘들어요. 수업 집중도 안 되고요. 흑흑,우린 시험 끝나고 행사가 너무 많아서 계속 노는 중이에요.,와 부러워요. 놀고 싶어요.,수련회 끝나면 모의고사 준비해야 돼요.
4,목 캔디 얼마죠?,700원이요.,그런가요? 응사할 때 목 캔디 드시는 분도 있다기에 나눔 해볼까 했는데 가격 멋지군요.,어…. 곽으로 든 게 700원이고요. 낱개 포장된 건 한 3만 원어치 샀다가 목요일...,그러셨군요. 가격은 3만 원 정도면 되는군요. 알려주셔서 감사해요,정확한 게 아니라. 큰 봉지 여러 개 든 거로 6봉지 가져갔었어요.,그러셨군요. 6봉지나,그렇게 많을 줄 몰라서.. 세네 봉지만 사셔도 될 거예요.


In [25]:
Aihub_v1 = AI_hub_tweater_talk.iloc

In [30]:
AI_hub_tweater_talk.drop_duplicates([0], inplace = True)

In [34]:
AI_hub_tweater_talk.reset_index(drop=True, inplace = True)

In [35]:
AI_hub_tweater_talk

,0,1,2,3,4,5,6,7
0,기계식 키보드 써보고 싶어요.,싼 것도 많던데 써보세요. 정말 좋아요.,싼 것도 있나요? 예전에 봤을 땐 다 10만 원이 넘던데요.,이거 쓰는데 예쁘고 딸깍거려요.,우와 진짜 예뻐요.,불 끄면 더 예뻐요.,저런 키보드 처음 봐요. 진짜 예쁘네요.,NaN
1,사실 여러 가지 글씨체가 있긴 한데 요즘 글씨를 안 쓰다 보니까 너무 맛 가버렸어요...,글씨 예쁘네요.,아니에요. 요즘 제 글씨 뭔지 너무 모르겠어요.,일정하지 않아도 저 정도면 너무 예쁜 거 아니에요? 난 악필이라서 노트 필기해도 볼...,나 노트필기 진짜 못해요. 맨날 하다가 포기해요.,저도 쓰다 보면 너무 구려서 포기하게 돼요. 그리고 공부도 포기하고요.,"맞아요. 아, 노트 필기해야지 하는데 글씨체 마음에 안 들면 바로 포기해요.",NaN
2,목 캔디 얼마죠?,700원이요.,그런가요? 응사할 때 목 캔디 드시는 분도 있다기에 나눔 해볼까 했는데 가격 멋지군요.,어…. 곽으로 든 게 700원이고요. 낱개 포장된 건 한 3만 원어치 샀다가 목요일...,그러셨군요. 가격은 3만 원 정도면 되는군요. 알려주셔서 감사해요,정확한 게 아니라. 큰 봉지 여러 개 든 거로 6봉지 가져갔었어요.,그러셨군요. 6봉지나,그렇게 많을 줄 몰라서.. 세네 봉지만 사셔도 될 거예요.
3,수요미식회 민호 실화에요?,다음 주요?,네 그런가 봐요! 예고 편에 나왔나 봐요.,우리 민호 맛난 거 먹으러 갔군요! 요즘만 같으면 애들 안정된 느낌이고 좋네요.,네! 다들 자리에서 바쁘네요. 그렇지만 같이 밥도 먹고 글도 남기기도 하고 좋아요 ...,좋겠네요 그 파티도 가고 대체 얼마 만이에요…. 부럽네요. 누나 자리 구해서 너무 ...,네 감사해요. 진짜 작년 손꼽게 본 것 같아요. 사는 게 뭔지 어휴 정말 사는 거 ...,NaN
4,여러분! 보험 가입할 때 선물로 뭐 받으면 좋을 거 같아요?,사은품 고르시는 건가요?,네. 보험 사은품이요.,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1529,욜로는 왜 갑자기 유행어가 되었는지 민망스러워요.,무한도전에서 나왔거든요. YOLO 하면서 막 카드 쓰고 돌아다녔어요.,"저도 그건 봤는데, 한국에서 욜로가 갑자기 유행해서 좀 의아했거든요. 티브이 프로그...","저도 돌아다니다 보면 가게 상호명에도 욜라가 눈에 많이 띄어서 약간 민망했는데, 무...","네. 좀 오래된 유행어인데, 왜 한국에서 갑자기 이렇게 붐이 일었는지 머리 말리면서...",그때도 외국에서 한물간 유형어를 가져다가 상호를 바꿀까요? 아니면 그전에 가게가 폐...,NaN,NaN
1530,"옥동식 인테리어 마음에 들어요. 특히 가게 위치와 크기, 배치는 내가 늘 꿈꾸던 것...",설거지하는 초보 막내에게 주 5일 하루 10시간 근무에 120만원 전후로 주면 현실...,먹어 보고 싶다는 느낌을 주는 식당이에요.,하지만 나는 돼지국밥 싫어하죠.,이렇게 말하면 흥분하는 사람 여럿 알아요.,NaN,NaN,NaN
1531,가게 인테리어가 예쁘네요.,드라이플라워네요.,가게 인테리어인데 드라이플라워가 참 많아요!,신기한 곳이네요. 근데 서울이 아니겠지요?,천안이에요. 집 근처에 있는 개인 가게에요!,"우와, 멋지네요. 멋진 가게 아닌가요?",멋진 가게에요. 음식도 맛있어요.,부럽네요. 양재는 회사 촌이라 맛있는 곳 찾기 힘들어요.
1532,식중독 조심하세요.,식중독 걸렸어요?,회사에서 먹은 두부가 이상했나 봐요. 후임도 탈났다는데 난 약 먹고 주사 맞고 괜찮...,주말에 고생했네요. 얼른 나으세요.,네. 항상 조심하세요. 카페 갈 거예요?,갈까 말까 고민 중이에요.,나도 고민 중인데 가게 되면 같이 가요.,좋아요. 그 때는 돈 많이 벌어가야겠어요.


In [36]:
AI_hub_tt_train = AI_hub_tweater_talk.iloc[:,:2]

In [39]:
AI_hub_tt_train.rename(columns={0:'Q',1:'A'}, inplace = True)

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [42]:
AI_hub_tt_train.to_csv('AI_hub_tt_train.csv')

# Ai hub V2

In [45]:
AI_hub_tweater_talk.head()

,0,1,2,3,4,5,6,7
0,기계식 키보드 써보고 싶어요.,싼 것도 많던데 써보세요. 정말 좋아요.,싼 것도 있나요? 예전에 봤을 땐 다 10만 원이 넘던데요.,이거 쓰는데 예쁘고 딸깍거려요.,우와 진짜 예뻐요.,불 끄면 더 예뻐요.,저런 키보드 처음 봐요. 진짜 예쁘네요.,NaN
1,사실 여러 가지 글씨체가 있긴 한데 요즘 글씨를 안 쓰다 보니까 너무 맛 가버렸어요...,글씨 예쁘네요.,아니에요. 요즘 제 글씨 뭔지 너무 모르겠어요.,일정하지 않아도 저 정도면 너무 예쁜 거 아니에요? 난 악필이라서 노트 필기해도 볼...,나 노트필기 진짜 못해요. 맨날 하다가 포기해요.,저도 쓰다 보면 너무 구려서 포기하게 돼요. 그리고 공부도 포기하고요.,"맞아요. 아, 노트 필기해야지 하는데 글씨체 마음에 안 들면 바로 포기해요.",NaN
2,목 캔디 얼마죠?,700원이요.,그런가요? 응사할 때 목 캔디 드시는 분도 있다기에 나눔 해볼까 했는데 가격 멋지군요.,어…. 곽으로 든 게 700원이고요. 낱개 포장된 건 한 3만 원어치 샀다가 목요일...,그러셨군요. 가격은 3만 원 정도면 되는군요. 알려주셔서 감사해요,정확한 게 아니라. 큰 봉지 여러 개 든 거로 6봉지 가져갔었어요.,그러셨군요. 6봉지나,그렇게 많을 줄 몰라서.. 세네 봉지만 사셔도 될 거예요.
3,수요미식회 민호 실화에요?,다음 주요?,네 그런가 봐요! 예고 편에 나왔나 봐요.,우리 민호 맛난 거 먹으러 갔군요! 요즘만 같으면 애들 안정된 느낌이고 좋네요.,네! 다들 자리에서 바쁘네요. 그렇지만 같이 밥도 먹고 글도 남기기도 하고 좋아요 ...,좋겠네요 그 파티도 가고 대체 얼마 만이에요…. 부럽네요. 누나 자리 구해서 너무 ...,네 감사해요. 진짜 작년 손꼽게 본 것 같아요. 사는 게 뭔지 어휴 정말 사는 거 ...,NaN
4,여러분! 보험 가입할 때 선물로 뭐 받으면 좋을 거 같아요?,사은품 고르시는 건가요?,네. 보험 사은품이요.,NaN,NaN,NaN,NaN,NaN


In [62]:
AI_hub_tt_train_v2 = AI_hub_tweater_talk.iloc[:,2:4]

In [63]:
AI_hub_tt_train_v2.dropna(subset=[3],inplace = True)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
AI_hub_tt_train_v2.reset_index(drop=True, inplace = True)

In [65]:
AI_hub_tt_train_v2.rename(columns={2:'Q',3:'A'},inplace = True)

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [66]:
AI_hub_tt_train_v2.to_csv('AI_hub_tt_train_v2.csv')

# Ai hub V3

In [61]:
AI_hub_tt_train_v3 = AI_hub_tweater_talk.iloc[:,4:6]

In [69]:
AI_hub_tt_train_v3.dropna(subset=[4,5],inplace = True)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [71]:
AI_hub_tt_train_v3.reset_index(drop=True, inplace = True)

In [72]:
AI_hub_tt_train_v3.rename(columns={4:'Q',5:'A'},inplace = True)

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [74]:
AI_hub_tt_train_v3.to_csv('AI_hub_tt_train_v3.csv')

# Ai hub -> 문장 단축, 호칭 제거

In [392]:
data = pd.read_csv('Chatbot_data/twit_data2.csv')

In [393]:
data = data.loc[:,['Q','A']]

In [194]:
import re
text = re.sub('[?님]','',text)

In [ ]:
re.findall(r'\w+?데[\b\W]',data['A'][0])

In [354]:
re.findall(r'\w+?데',data['A'][0])[0]

'많던데'

In [394]:
newQ = []
newA = []

In [ ]:
data['Q'] = data['Q'].astype('str')

In [395]:
for text in data['A']:
    
    try:
        a = re.findall(r'\w+?님', text)[0]
        #print(a)
        if a[0] != '':
            new = re.sub(a,'',text)
            new.strip()
            #print(new)
            newA.append(new)
        
    except:
        newA.append(text)
    

In [396]:
for text in data['Q']:
    
    try:
        a = re.findall(r'\w+?님', text)[0]
        #print(a)
        if a[0] != '':
            new = re.sub(a,'',text)
            new.strip()
            #print(new)
            newQ.append(new)
        
    except:
        newQ.append(text)
    

In [397]:
len(newA),len(newQ)

(1534, 1534)

In [410]:
newData = pd.DataFrame(newQ, columns={'Q'})

In [420]:
newData.drop_duplicates('Q',inplace = True)

In [449]:
newData.to_csv('Chatbot_data/newAI_hub_train.csv')

In [459]:
a = pd.read_csv('Chatbot_data/newAI_hub_train.csv')

In [456]:
a.dropna(inplace = True)

In [458]:
a.to_csv('Chatbot_data/newAI_hub_train.csv',index = False)

In [460]:
len(a)

1472

In [424]:
def length(series):
    
    return len(series)
    

In [427]:
newData['length'] = newData['Q'].astype('str').apply(length)

In [430]:
newData['length'].

25.127574750830565

In [431]:
newData.describe()

,length
count,1505.000000
mean,25.127575
std,18.069854
min,1.000000
25%,13.000000
50%,20.000000
75%,31.000000
max,146.000000


In [472]:
newData = newData.loc[:,['Q','A']]

In [497]:
ogData = pd.read_csv('Chatbot_data/train.csv', encoding='CP949')

In [470]:
ogData = ogData.loc[:,['Q','A']]

In [477]:
ogData.drop_duplicates('Q',inplace = True)

In [498]:
totalData = pd.concat([ogData,newData])

In [499]:
len(totalData)

23968

In [483]:
totalData.dropna(inplace = True)

In [485]:
totalData.to_csv('Chatbot_data/total_train.csv')

In [487]:
testData = pd.read_csv('Chatbot_data/test.csv',encoding='CP949')

In [488]:
len(testData)

1183

In [513]:
totalData = pd.concat([totalData, testData])

In [514]:
len(totalData)

25151

In [520]:
totalData = totalData.loc[:,['Q','A']]

In [524]:
totalData.isnull().sum()

Q     1
A    32
dtype: int64

In [525]:
len(totalData)

25151

In [527]:
totalData.dropna(inplace = True)

In [528]:
totalData.to_csv('Chatbot_data/total_train.csv')

In [530]:
Data = pd.read_csv('Chatbot_data/total_train.csv')

In [2]:
online = pd.read_csv('Chatbot_data/온라인_댓글_데이터셋(3).csv',header=None)

In [5]:
online1000 = online.iloc[:1000]

In [6]:
online10 = online.iloc[:10]

In [7]:
online10

,0
0,"그래 대통령 욕은 왜하는겨, 정녕 욕먹을 사람은 선장이랑 이후 대처를 제대로 하지못..."
1,시바 잘하는 짓거리다 가관이네 가관
2,누가되도 이보단 최악일수없다...
3,근데 솔까 항상 청문회때 저런 질문들 하든데.. 걍 무뇌인거 입증하는거 뿐이 안됨..
4,이래서 진짜 국민 무소속이 대통령이 되야함
5,얼마전에 본 팀버튼의 이상한 나라의 앨리스 영화 생각납니다. 못된 여왕은 큰 얼굴이...
6,하는 꼬라지 보니 별로 기대할 것도 없수다! 그게 속 편하겠수다. 그 늙어빠진 선장...
7,차라리 지난번 해양경찰청장 이강덕이 좋은듯
8,거기 왜 나갔어요?
9,네이버 운영하는 놈들도 조작녀 박그네 후장빠는 골빈 무뇌아들인가봐 그러니 네이버가 ...


# Quizfinal(깡통이영어선생님)

In [41]:
quiz = pd.read_csv('Chatbot_data/eng_quiz_data_v2.0.csv')

In [12]:
len(quiz['A'][0].split(','))

2

In [13]:
a = []

for i in quiz['A']:
    a.append(len(i.split(',')))

In [20]:
a = pd.DataFrame(a)

In [23]:
a[0].value_counts()

1    799
2      1
Name: 0, dtype: int64

In [46]:
quiz = quiz.drop(0).reset_index(drop = True)

In [57]:
print(quiz['Q'][5])

점심식사 후에 그는 서재로 물러갔다. 
After lunch, he retired to his study.

live, keep, supper, after


In [67]:
for i in range(len(quiz['Q'])):
    if len(quiz['Q'][i].split('_______')) < 2:
        print(quiz['Q'][i])
        quiz.drop(i, inplace = True)

점심식사 후에 그는 서재로 물러갔다. 
After lunch, he retired to his study.

live, keep, supper, after
적당히 대답을 해 주십시오 
Answer him as you think fit.

answer, dial, about, only
공상 과학 소설은 아주 재미있기 때문이야. 
Because science fiction is very fun.

because, lesson, cassette, poor
한강 대교는 길이가 얼마나 됩니까? 
How long is the Han River Bridge?

market, bridge, stupid, together
이 자켓을 입어봐도 돼요? 
Can I try on this jacket?

red, fair, woman, can
공이 다시 튀어올 때 잡아라. 
Catch the ball on the rebound.

cold, catch, run, afternoon
푸른색 깃발을 가져다 주세요. 
Bring the blue flag to me.

lip, stand, cross, wide
파이를 반으로 잘라 맛있게 먹는다. 
Cut the pie in half and enjoy it.

cut, show, present, fair
넌 내 딸이 아니야. 
You are not my daughter.

hear, now, tell, daughter 
실물 그대로 그려라. 
Draw a picture true to nature.

touch, wing, draw, across
모든 만화 영화는 생각에서 시작됩니다
Every cartoon begins with an idea.

salt, every, lake, post
아주 재미있는 댄스 게임이야. 
It's a very exciting dance game.

key, clock, dollar, excite
그릇을 얼음물로 채워라. 
Fill a bowl with ice water.

fill, guitar, dark, pa

In [71]:
quiz.reset_index(inplace = True)

In [72]:
for i in range(len(quiz['Q'])):
    if len(quiz['Q'][i].split('_______')) < 2:
        print(quiz['Q'][i])

In [93]:
quiz.drop('index', axis=1,inplace = True)

In [94]:
quiz.to_csv('Chatbot_data/quizfinal.csv')

# 바퀴데이터셋 

In [9]:
baqui = pd.read_excel('kkangtongData/한국어_연속적_대화_데이터셋.xlsx', engine='openpyxl', header=1, usecols='A:C')

In [12]:
baqui.rename(columns={'dialog #':'start','발화':'dialog','감정':'emotion'},inplace = True)

In [16]:
baqui_start = baqui[baqui['start']=='S'] # 발화만 저장

In [20]:
bs_index = baqui_start.index # 발화인덱스

In [37]:
j = 0 
start = []
end = []
for i in bs_index:
    if baqui['emotion'][i+1] in ['분노','혐오']:
        start.append(baqui['dialog'][i])
        end.append(baqui['dialog'][i+1])

df = {'Q':start,'A':end}
df_baqui = pd.DataFrame(data = df)

## 첫발화 다음 분노 혐오 데이터만 저장

In [ ]:

start = []
end = []
for i in range(len(baqui.emotion)):
    if baqui.emotion[i+1] in ['분노','혐오']:
        if baqui['start'][i+1] != 'S':
            start.append(baqui.dialog[i])
            end.append(baqui.dialog[i+1])
df = {'Q':start,'A':end}

df_baqui_2 = pd.DataFrame(data = df)
    

In [51]:
df = {'Q':start,'A':end}

df_baqui_2 = pd.DataFrame(data = df)
    

In [96]:
df_baqui_sum = pd.concat([df_baqui, df_baqui_2])

In [97]:
df_baqui_sum.reset_index(drop=True,inplace = True)

In [109]:
def split_dialog(dialog):
    if len(dialog) > 50:
        return dialog.split('.')[0]
    else:
        return dialog

In [134]:
df_baqui_sum['Q'] = df_baqui_sum['Q'].apply(lambda x : x.split('.')[0] if len(x) > 40 else x) # 람다로 지우기

In [110]:
df_baqui_sum['Q'] = df_baqui_sum['Q'].apply(split_dialog)
df_baqui_sum['A'] = df_baqui_sum['A'].apply(split_dialog)

In [71]:
def length(feature):
    a = len(feature)
    return a   

In [113]:
df_baqui_sum['Qlen'] = df_baqui_sum['Q'].apply(length)
df_baqui_sum['Alen'] = df_baqui_sum['A'].apply(length)

In [102]:
df_baqui_sum.describe() # 원래 길이

,Qlen,Alen
count,3671.000000,3671.000000
mean,25.630074,31.438300
std,19.991113,24.186231
min,1.000000,1.000000
25%,12.000000,15.000000
50%,21.000000,25.000000
75%,33.000000,40.500000
max,168.000000,200.000000


In [115]:
df_baqui_sum.describe() # 정제후 길이

,Qlen,Alen
count,3671.000000,3671.000000
mean,21.441297,25.453827
std,14.013706,16.639190
min,0.000000,0.000000
25%,11.000000,13.000000
50%,19.000000,22.000000
75%,30.000000,35.000000
max,133.000000,159.000000


In [121]:
# 긴 문장 데이터 삭제
df_baqui_sum.reset_index(drop=True, inplace = True)

for i in range(len(df_baqui_sum['Qlen'])):
    if df_baqui_sum['Qlen'][i] >= 50:
        df_baqui_sum.drop(i,inplace = True)
        
for i in range(len(df_baqui_sum['Alen'])):
    if df_baqui_sum['Alen'][i] >= 50:
        df_baqui_sum.drop(i,inplace = True)

In [123]:
df_baqui_sum.describe() # 정제후 길이

,Qlen,Alen
count,3326.000000,3326.000000
mean,20.205652,22.430848
std,11.845767,11.983492
min,0.000000,0.000000
25%,11.000000,13.000000
50%,18.000000,21.000000
75%,29.000000,32.000000
max,49.000000,49.000000


In [127]:
df_baqui_sum.reset_index(drop=True, inplace = True)

# Df_baqui_sum

In [128]:
df_baqui_sum.to_csv('Chatbot_train_data/df_baqui_sum.csv')